In [ ]:
import pandas as pd
import pysam
from Bio.Seq import Seq
import re

# Define file paths
gtf_path = "/global/scratch/users/sergiomar10/data/GCF_000001405.40_GRCh38.p14_genomic.gtf"
vcf_path = "/global/scratch/users/sergiomar10/data/clinvar_20210302.vcf.gz"
fasta_path = "/global/scratch/users/sergiomar10/data/GCF_000001405.40_GRCh38.p14_genomic.fna"

# -----------------------------------------------------
# 1. Load and parse the GTF file with pandas
gtf_cols = ["chrom", "source", "feature", "start", "end", "score", "strand", "frame", "attributes"]
gtf_df = pd.read_csv(gtf_path, sep="\t", comment='#', header=None, names=gtf_cols)

# Keep only CDS features
cds_df = gtf_df[gtf_df['feature'] == 'CDS'].copy()

# Function to extract transcript_id from the attributes column
def extract_transcript_id(attr):
    match = re.search('transcript_id "([^"]+)"', attr)
    if match:
        return match.group(1)
    return None

cds_df['transcript_id'] = cds_df['attributes'].apply(extract_transcript_id)

# Group CDS features by transcript
# We build a dictionary: transcript_id -> list of exons (each as a dict with chrom, start, end, strand)
cds_by_transcript = {}
for transcript, group in cds_df.groupby("transcript_id"):
    # Use the strand from the first row (assumed consistent across exons)
    strand = group.iloc[0]['strand']
    # For plus strand, sort exons by start; for minus, sort in reverse genomic order
    if strand == "+":
        group_sorted = group.sort_values("start")
    else:
        group_sorted = group.sort_values("start", ascending=False)
    exons = []
    for _, row in group_sorted.iterrows():
        exon = {
            "chrom": row["chrom"],
            "start": int(row["start"]),
            "end": int(row["end"]),
            "strand": row["strand"]
        }
        exons.append(exon)
    cds_by_transcript[transcript] = exons

# -----------------------------------------------------
# 2. Helper functions to rebuild the CDS sequence and find overlapping transcripts

def get_transcript_cds_with_mapping(transcript_id, exons, fasta):
    """
    Given a transcript's exons (list of dicts with chrom, start, end, strand)
    and an open pysam FASTA file, return:
      - full CDS sequence as a string,
      - a mapping list where each CDS index maps to (chrom, genomic_position),
      - the strand.
    """
    strand = exons[0]["strand"]
    cds_seq = ""
    mapping = []
    for exon in exons:
        chrom = exon["chrom"]
        start = exon["start"]
        end = exon["end"]
        # pysam FASTA is 0-based so subtract 1 from start
        seq = fasta.fetch(chrom, start - 1, end)
        if strand == "-":
            # For the minus strand, reverse complement the exon sequence
            seq = str(Seq(seq).reverse_complement())
            # Build mapping in reverse order
            positions = [(chrom, pos) for pos in range(end, start - 1, -1)]
        else:
            positions = [(chrom, pos) for pos in range(start, end + 1)]
        cds_seq += seq
        mapping.extend(positions)
    return cds_seq, mapping, strand

def get_transcripts_overlapping_variant(chrom, pos, cds_by_transcript):
    """
    Return a list of transcript IDs where the variant (chrom, pos) falls within at least one CDS exon.
    """
    overlapping = []
    for transcript, exons in cds_by_transcript.items():
        for exon in exons:
            if exon["chrom"] == chrom and exon["start"] <= pos <= exon["end"]:
                overlapping.append(transcript)
                break  # if one exon overlaps, no need to check further for this transcript
    return overlapping


: 

: 

In [ ]:
# -----------------------------------------------------
# 3. Open FASTA and VCF files using pysam
fasta = pysam.FastaFile(fasta_path)

In [2]:
vcf = pysam.VariantFile(vcf_path)

records = []
variant_count = 0  # To count the number of processed variants

# Map from common numeric/letter chromosome IDs (as found in VCF) 
# to the full NCBI contig names in the FASTA
chrom_map = {
    "1":  "NC_000001.11",
    "2":  "NC_000002.12",
    "3":  "NC_000003.12",
    "4":  "NC_000004.12",
    "5":  "NC_000005.10",
    "6":  "NC_000006.12",
    "7":  "NC_000007.14",
    "8":  "NC_000008.11",
    "9":  "NC_000009.12",
    "10": "NC_000010.11",
    "11": "NC_000011.10",
    "12": "NC_000012.12",
    "13": "NC_000013.11",
    "14": "NC_000014.9",
    "15": "NC_000015.10",
    "16": "NC_000016.10",
    "17": "NC_000017.11",
    "18": "NC_000018.10",
    "19": "NC_000019.10",
    "20": "NC_000020.11",
    "21": "NC_000021.9",
    "22": "NC_000022.11",
    "X":  "NC_000023.11",
    "Y":  "NC_000024.10",
    "MT": "NC_012920.1",  # Mitochondrial DNA
}

In [13]:
variant_count = 0  # To count the number of processed variants
records = []

for x, rec in enumerate(vcf.fetch()):
    # Filter based on clinical significance: only process variants that are Pathogenic or Benign.
    # (Assumes that the 'CLNSIG' field is present and contains strings like "Pathogenic" or "Benign")
    clnsig = rec.info.get('CLNSIG')
    if clnsig is None or not any(sig in {"Pathogenic", "Benign"} for sig in clnsig):
        continue

    # print(rec.chrom, rec.pos, rec.ref, rec.alts, clnsig)

    chrom = rec.chrom
    pos  = rec.pos
    ref = rec.ref
    alts = rec.alts

    # Map chromosome names if needed
    if chrom in chrom_map:
        chrom = chrom_map[chrom]
    
    transcripts = get_transcripts_overlapping_variant(chrom, pos, cds_by_transcript)

    if not transcripts:
        continue  # Skip non-coding variants

    # Process each overlapping transcript
    for transcript in transcripts:
        exons = cds_by_transcript[transcript]
        cds_seq, mapping, strand = get_transcript_cds_with_mapping(transcript, exons, fasta)
        
        # Confirm the variant's genomic coordinate is part of the CDS mapping
        if (chrom, pos) not in mapping:
            continue

        cds_index = mapping.index((chrom, pos))
        try:
            # Construct the mutated CDS sequence.
            # Note: This example assumes a simple SNP (i.e. ref and alt of length 1).
            mutated_seq = cds_seq[:cds_index] + rec.alts[0] + cds_seq[cds_index + 1:]
            
            # Translate the CDS sequences into proteins.
            ref_protein = str(Seq(cds_seq).translate(to_stop=False))
            mut_protein = str(Seq(mutated_seq).translate(to_stop=False))

            if len(mut_protein) < 10:
                break
            
            residue_index = cds_index // 3


            record = {
                "chrom": chrom,
                "pos": pos,
                "transcript": transcript,
                "mutated_residue": residue_index,
                "cds_index": cds_index,
                "cds_seq": ref_protein,
                "mutated_seq": mut_protein,
                "strand": strand,
                "clnsig": clnsig,
            }
            records.append(record)

            variant_count += 1

            if x > 30000:
                break
        except:
            print("Error processing variant at")

print("Processed", variant_count, "variants")
print("Recorded", len(records), "coding variants with relevant clinical significance.")

/clusterfs/nilah/sergio/miniconda3/envs/ESM_cambrian/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing variant at
Error processing var

KeyboardInterrupt: 

In [15]:
records_df = pd.DataFrame(records)

In [17]:
records_df.to_csv("coding_variants_30k.csv", index=False)

In [6]:
records_df = pd.DataFrame(records)

In [16]:
records_df

,chrom,pos,transcript,mutated_residue,cds_index,cds_seq,mutated_seq,strand,clnsig
0,NC_000001.11,939398,NM_001385640.1,394,1183,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,+,"(Benign,)"
1,NC_000001.11,939398,NM_001385641.1,393,1180,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,+,"(Benign,)"
2,NC_000001.11,939398,NM_152486.4,214,643,MSKGILQVHPPICDCPGCRISSPVNRGRLADKRTVALPAARNLKKE...,MSKGILQVHPPICDCPGCRISSPVNRGRLADKRTVALPAARNLKKE...,+,"(Benign,)"
3,NC_000001.11,943995,NM_001385640.1,793,2379,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,+,"(Pathogenic,)"
4,NC_000001.11,943995,NM_001385641.1,792,2376,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,+,"(Pathogenic,)"
...,...,...,...,...,...,...,...,...,...
36386,NC_000001.11,119917707,NM_024408.4,1994,5984,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,-,"(Pathogenic,)"
36387,NC_000001.11,119918491,NM_024408.4,1947,5843,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,-,"(Pathogenic,)"
36388,NC_000001.11,119919428,NM_024408.4,1888,5664,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,-,"(Pathogenic,)"
36389,NC_000001.11,119919449,NM_024408.4,1881,5643,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,-,"(Pathogenic,)"
